# **Google Capstone Project: #Case 1: Cyclistic**

**Case Study: How Does a Bike-Share Navigate Speedy Success?**

# **Overview**

**Objective:** Increase revenue by attracting causual members switching to annual membership. 

**Business Task:** Study the dataset from the last 12 months to gain insight about the usage pattern between annual members and causal members.

**Steps:** Using six steps of data analysis process: ask, prepare, process, analyze, share ,act  

# **Ask**

1.How do annual members and casual riders use Cyclistic bikes differently?

# **Prepare**
**Divvy Data**
https://ride.divvybikes.com/system-data

# **Process**
**Tools:** Excel, R, Tableau

# **Data Explorations**

In [1]:
#Import all the library
library(tidyverse)
library(lubridate)
library(zoo)
library(skimr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union



Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric




In [2]:
#Setting directory on Kaggle
setwd("../input/divvy-tripdata-nov-2020-nov-2021")

In [3]:
#Check directory for reading dataset is correct
getwd()

[1] "/kaggle/input/divvy-tripdata-nov-2020-nov-2021"

In [4]:
#Force all the columns to character type due to different data type unable to import. Import CSV files.
temp=list.files(pattern = "*.csv") %>% 
map_df(~read_csv(., col_types = cols(.default = "c")))
df=data.frame(temp)

In [5]:
#Overview for the dataset (Rows: 5,738,812,columns: 13) 
glimpse(df)

Rows: 5,738,812
Columns: 13
$ ride_id            <chr> "BD0A6FF6FFF9B921", "96A7A7A4BDE4F82D", "C61526D065…
$ rideable_type      <chr> "electric_bike", "electric_bike", "electric_bike", …
$ started_at         <chr> "2020-11-01 13:36:00", "2020-11-01 10:03:26", "2020…
$ ended_at           <chr> "2020-11-01 13:45:40", "2020-11-01 10:14:45", "2020…
$ start_station_name <chr> "Dearborn St & Erie St", "Franklin St & Illinois St…
$ start_station_id   <chr> "110", "672", "76", "659", "2", "72", "76", NA, "58…
$ end_station_name   <chr> "St. Clair St & Erie St", "Noble St & Milwaukee Ave…
$ end_station_id     <chr> "211", "29", "41", "185", "2", "76", "72", NA, "288…
$ start_lat          <chr> "41.8941765", "41.89095866666667", "41.880982833333…
$ start_lng          <chr> "-87.62912733333333", "-87.63534283333334", "-87.61…
$ end_lat            <chr> "41.89443416666667", "41.900675", "41.8720545", "41…
$ end_lng            <chr> "-87.62337916666667", "-87.66248033333333", "-87.62…
$ member_cas

In [6]:
#Display first 6 rows
head(df)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,BD0A6FF6FFF9B921,electric_bike,2020-11-01 13:36:00,2020-11-01 13:45:40,Dearborn St & Erie St,110,St. Clair St & Erie St,211,41.8941765,-87.62912733333333,41.89443416666667,-87.62337916666667,casual
2,96A7A7A4BDE4F82D,electric_bike,2020-11-01 10:03:26,2020-11-01 10:14:45,Franklin St & Illinois St,672,Noble St & Milwaukee Ave,29,41.89095866666667,-87.63534283333334,41.900675,-87.66248033333333,casual
3,C61526D06582BDC5,electric_bike,2020-11-01 00:34:05,2020-11-01 01:03:06,Lake Shore Dr & Monroe St,76,Federal St & Polk St,41,41.880982833333334,-87.61675416666667,41.8720545,-87.62955033333333,casual
4,E533E89C32080B9E,electric_bike,2020-11-01 00:45:16,2020-11-01 00:54:31,Leavitt St & Chicago Ave,659,Stave St & Armitage Ave,185,41.89549916666667,-87.682013,41.9177445,-87.69139183333333,casual
5,1C9F4EF18C168C60,electric_bike,2020-11-01 15:43:25,2020-11-01 16:16:52,Buckingham Fountain,2,Buckingham Fountain,2,41.87649733333333,-87.620358,41.876448333333336,-87.620338,casual
6,7259585D8276D338,electric_bike,2020-11-14 15:55:17,2020-11-14 16:44:38,Wabash Ave & 16th St,72,Lake Shore Dr & Monroe St,76,41.860288833333335,-87.625806,41.880985,-87.6167735,casual


In [7]:
#Check for duplicated records
sum(duplicated(df))

[1] 0

In [8]:
#Check for number of null records
skim(df)

── Data Summary ────────────────────────
                           Values 
Name                       df     
Number of rows             5738812
Number of columns          13     
_______________________           
Column type frequency:            
  character                13     
________________________          
Group variables            None   

── Variable type: character ────────────────────────────────────────────────────
   skim_variable      n_missing complete_rate   min   max empty n_unique
 1 ride_id                    0         1        16    16     0  5738603
 2 rideable_type              0         1        11    13     0        3
 3 started_at                 0         1        19    19     0  4807889
 4 ended_at                   0         1        19    19     0  4799134
 5 start_station_name    675769         0.882     3    53     0      846
 6 start_station_id      675876         0.882     1    36     0     1305
 7 end_station_name      725658         0.874    10

# **Data Manipulations & Data Cleaning**

Data cleaning 

In [9]:
#Data manipulations for started_at is greater than ended_at
df1=df%>%
  mutate(new_started_at=ifelse(started_at>ended_at,ended_at,started_at))
  df1$new_started_at=as_datetime(df1$new_started_at)
df2=df%>%
  mutate(new_ended_at=ifelse(ended_at<started_at,started_at,ended_at))
  df1$new_ended_at=as_datetime(df2$new_ended_at)

In [10]:
#Check manipulation is done correctly
filter(df1,ride_id=="840F04F527426550")

ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,new_started_at,new_ended_at
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dttm>,<dttm>
840F04F527426550,docked_bike,2020-11-25 15:13:48,2020-11-25 15:19:09,Racine Ave & Belmont Ave,226,Halsted St & Roscoe St,299,41.939743,-87.658865,41.94367,-87.64895,member,2020-11-25 15:13:48,2020-11-25 15:19:09
840F04F527426550,docked_bike,2020-12-15 11:53:55,2020-11-25 15:19:09,Racine Ave & Belmont Ave,TA1308000019,Halsted St & Roscoe St,TA1309000025,41.939743,-87.658865,41.94367,-87.64895,member,2020-11-25 15:19:09,2020-12-15 11:53:55


In [11]:
#Select columns relevant to initial analysis.
#df1=select(df1,-start_station_id,-end_station_id,-(start_lat:end_lng))

In [12]:
#Convert back to datetime since force all columns to character during import
df1$started_at=as_datetime(df1$started_at)
df1$ended_at=as_datetime(df1$ended_at)

In [13]:
#Calculate ride_length and convert it to minutes
df1$ride_length=as.numeric(round((abs(df1$new_ended_at-df1$new_started_at)/60),2))

In [14]:
#Convert stated_at to weekday
df1$weekday <- factor(weekdays(df1$started_at))

In [15]:
#Convert started_at to month
df1$month <- format(df1$started_at, "%m")

In [16]:
#Convert started_at to season
df1$season=factor(df1$month,labels=c("Winter","Winter","Spring","Spring","Spring","Summer","Summer","Summer","Fall","Fall","Fall","Winter"))

# **Analyze**

**Data Analysis**

In [17]:
#Find the average of ride length
mean(df1$ride_length)

[1] 23.90064

In [18]:
#Find the maximum ride length
max(df1$ride_length)

[1] 55944.15

In [19]:
#Find the mean and standard deviation between member_casual and ride_length
summarise(group_by(df1,member_casual),Result=mean(ride_length),sd(ride_length))

member_casual,Result,sd(ride_length)
<chr>,<dbl>,<dbl>
casual,33.03821,306.0890
member,16.45083,279.3143


In [20]:
#Find the mean and standard deviation between member_casual, weekday and ride_length
summarise(group_by(df1,weekday,member_casual),Result=mean(ride_length),sd(ride_length))

`summarise()` has grouped output by 'weekday'. You can override using the `.groups` argument.



weekday,member_casual,Result,sd(ride_length)
<fct>,<chr>,<dbl>,<dbl>
Friday,casual,30.56854,284.70750
Friday,member,13.48208,27.52701
Monday,casual,31.81678,214.89899
Monday,member,13.28577,28.17449
Saturday,casual,34.93903,282.52078
Saturday,member,15.35883,30.04077
Sunday,casual,37.77527,301.71973
Sunday,member,15.70626,32.08099
Thursday,casual,27.90490,252.53637


In [21]:
#Find number of bike riders by weekday
summarise(group_by(df1,member_casual,weekday),Result=length(ride_id))

`summarise()` has grouped output by 'member_casual'. You can override using the `.groups` argument.



member_casual,weekday,Result
<chr>,<fct>,<int>
casual,Friday,367991
casual,Monday,293067
casual,Saturday,572444
casual,Sunday,493641
casual,Thursday,288536
casual,Tuesday,280489
casual,Wednesday,281278
member,Friday,455356
member,Monday,436437


In [22]:
#Find number of bike riders by season
summarise(group_by(df1,season),Result=length(ride_id))

season,Result
<fct>,<int>
Winter,278029
Spring,1097359
Summer,2356357
Fall,2007067


In [23]:
#Find the mean and standard deviation between season and ride_length
summarise(group_by(df1,season,member_casual),Result=mean(ride_length),sd(ride_length))


`summarise()` has grouped output by 'season'. You can override using the `.groups` argument.



season,member_casual,Result,sd(ride_length)
<fct>,<chr>,<dbl>,<dbl>
Winter,casual,69.03872,1078.39348
Winter,member,52.63580,1054.39772
Spring,casual,38.15876,299.28374
Spring,member,14.49916,26.42399
Summer,casual,32.75205,280.04214
Summer,member,14.33053,26.64429
Fall,casual,27.89891,206.09566
Fall,member,12.80465,26.54910


In [24]:
#Find type of bike riders
summarise(group_by(df1,member_casual),Result=length(ride_id))

member_casual,Result
<chr>,<int>
casual,2577446
member,3161366


# **Export to CSV file**

In [25]:
#Export to CSV file
#write.csv(df1,"C:/Users/Kent/Desktop/Google Data Analytics/Capstone Project/Case Study 1/Exported CSV file/bike_rider_capstone_project.csv")

# **Share**
Import into Tableau for further analysis and Tableau**

https://public.tableau.com/app/profile/kent.tam/viz/CapstoneProjectPractice/CapstoneProject1

# **Act**

Provide insights and recommendations for Stakeholders.

# **Reference**

Prepare
Divvy Data https://ride.divvybikes.com/system-data

Coursera Capstone Project Scope